In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/00


In [2]:
f = r.TFile("/home/jlab/g-2/omega_a_KS/data/results_clustersAndCoinc_unrandomized_July7_pileup_corrected.root")
h = f.Get("clustersAndCoincidences/corrected").Clone()

In [12]:
def rootHistToNumpy(hist3d):
    
    check_hist_cpp = '''
    int checkHist(std::size_t size, const TH3* hist) 
    {
        if (hist == nullptr) 
        {
            return -1;
        }

        std::size_t nHistBins =
        hist->GetNbinsX() * hist->GetNbinsY() * hist->GetNbinsZ();
        if (nHistBins != size) 
        {
            return -2;
        }

        return 0;
    }

    '''
    r.gInterpreter.ProcessLine(check_hist_cpp)


    cpp_code = '''
        int hist3dToNumpyArray(double* data, double* error, std::size_t size, const char* histName)     {
            const TH3* hist = (TH3*)gROOT->FindObject(histName);
            int retcode = checkHist(size, hist);
            if (retcode != 0) 
            {
                return retcode;
            }

            for (int zBin = 1; zBin <= hist->GetNbinsZ(); ++zBin) 
            {
                for (int yBin = 1; yBin <= hist->GetNbinsY(); ++yBin) 
                {
                    for (int xBin = 1; xBin <= hist->GetNbinsX(); ++xBin) 
                    {
                        *data++ = hist->GetBinContent(xBin, yBin, zBin);
                        *error++ = hist->GetBinError(xBin, yBin, zBin);
                    }
                }
            }

            return 0;
        }
    '''
    r.gInterpreter.ProcessLine(cpp_code)
    
    
    shape = (hist3d.GetNbinsZ(), hist3d.GetNbinsY(), hist3d.GetNbinsX())
    array = np.empty(shape)
    err_array = np.empty(shape)

    retcode = r.hist3dToNumpyArray(
        array, err_array, array.size, hist3d.GetName().encode())
    
    return array, err_array

In [14]:
%%time

array, err_array = rootHistToNumpy(h)

CPU times: user 6.62 s, sys: 3.95 s, total: 10.6 s
Wall time: 10.5 s


In [15]:
array[11,11,11]

6414.646020207356

In [16]:
err_array[11,11,11]

80.09148531652635